In [1]:
#Install Required Packages


!pip install -q chromadb pandas pdfplumber sentence-transformers gradio




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.9 MB/s eta 0:00

In [2]:
# Upload Property CSV
from google.colab import files
import pandas as pd

print("📤 Please upload your lebanon_properties.csv file:")
uploaded = files.upload()

# Load the data
csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(csv_filename)

print(f"\n✅ Loaded {len(df)} properties")
print(f"📊 Columns: {list(df.columns)}")
print(f"\n📋 Sample property:")
print(df.head(1).T)  # Show first property transposed

📤 Please upload your lebanon_properties.csv file:


Saving lebanon_properties_sample.csv to lebanon_properties_sample.csv

✅ Loaded 200 properties
📊 Columns: ['property_id', 'title', 'property_type', 'district', 'governorate', 'price_usd', 'size_sqm', 'price_per_sqm', 'bedrooms', 'bathrooms', 'features', 'listing_date', 'price_category']

📋 Sample property:
                                              0
property_id                             LB00001
title                    Apartment in Achrafieh
property_type                         apartment
district                              Achrafieh
governorate                              Beirut
price_usd                                305433
size_sqm                                     85
price_per_sqm                           3593.33
bedrooms                                      3
bathrooms                                     1
features        ['parking', 'view', 'elevator']
listing_date                         2024-08-22
price_category                          Premium


In [3]:
# Upload RAMCO PDF Reports
import os

# Create folder for reports
os.makedirs('ramco_reports', exist_ok=True)

print("📤 Please upload your RAMCO PDF files:")
print("(You can select multiple files at once)")
uploaded_pdfs = files.upload()

# Save PDFs to folder
for filename in uploaded_pdfs.keys():
    filepath = f'ramco_reports/{filename}'
    with open(filepath, 'wb') as f:
        f.write(uploaded_pdfs[filename])

# List uploaded PDFs
import glob
pdf_files = glob.glob('ramco_reports/*.pdf')

print(f"\n✅ Uploaded {len(pdf_files)} PDF files")
print(f"\n📋 Files:")
for pdf in pdf_files:
    file_size = os.path.getsize(pdf) / 1024  # KB
    print(f"  • {os.path.basename(pdf)} ({file_size:.1f} KB)")

📤 Please upload your RAMCO PDF files:
(You can select multiple files at once)


Saving Q1Ramcoquarterly2[1].pdf to Q1Ramcoquarterly2[1].pdf
Saving Q2Ramcoquarterly2[1].pdf to Q2Ramcoquarterly2[1].pdf
Saving RAMCOTheQuarterlyQ[1].pdf to RAMCOTheQuarterlyQ[1].pdf
Saving RAMCOTheQuarterlyQ[2].pdf to RAMCOTheQuarterlyQ[2].pdf

✅ Uploaded 4 PDF files

📋 Files:
  • Q1Ramcoquarterly2[1].pdf (2052.0 KB)
  • RAMCOTheQuarterlyQ[1].pdf (4524.2 KB)
  • Q2Ramcoquarterly2[1].pdf (1906.1 KB)
  • RAMCOTheQuarterlyQ[2].pdf (6490.7 KB)


In [4]:
# Install OCR (Tesseract)
print("📦 Installing OCR tools (this takes 2-3 minutes)...")
print("=" * 70)

# Install Tesseract OCR
!apt-get install -y -qq tesseract-ocr > /dev/null 2>&1
!apt-get install -y -qq libtesseract-dev > /dev/null 2>&1

# Install Python packages
!pip install -q pytesseract pdf2image pillow

print("✅ OCR tools installed!")

# Test Tesseract
import pytesseract
from pdf2image import convert_from_path

print("✅ Ready to extract text from scanned PDFs!")

📦 Installing OCR tools (this takes 2-3 minutes)...
✅ OCR tools installed!
✅ Ready to extract text from scanned PDFs!


In [5]:
#  Install Poppler and Extract with OCR
print(" Installing Poppler...")
!apt-get install -y -qq poppler-utils > /dev/null 2>&1
print(" Poppler installed!")

from pdf2image import convert_from_path
import pytesseract
import os

def extract_text_with_ocr(pdf_path):
    """Extract text from scanned PDF using OCR"""
    try:
        filename = os.path.basename(pdf_path)
        print(f"  📄 Converting to images...")

        # Convert PDF to images (with poppler)
        images = convert_from_path(
            pdf_path,
            dpi=200,  # Lower DPI for faster processing
            fmt='jpeg'
        )
        print(f"   {len(images)} pages converted")

        # OCR each page
        print(f"  🔍 OCR processing...")
        full_text = ""

        for i, image in enumerate(images):
            print(f"    Page {i+1}/{len(images)}...", end=" ")
            page_text = pytesseract.image_to_string(image, lang='eng')
            full_text += page_text + "\n\n"


        return full_text

    except Exception as e:
        print(f"  ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return ""

# Process PDFs
pdf_files = glob.glob('ramco_reports/*.pdf')

print("\n📖 Extracting text with OCR (5-10 minutes)...")
print("=" * 70)

extracted_reports = []

for i, pdf_file in enumerate(pdf_files, 1):
    filename = os.path.basename(pdf_file)
    print(f"\n[{i}/{len(pdf_files)}] {filename}")

    text = extract_text_with_ocr(pdf_file)

    if text and len(text) > 100:
        extracted_reports.append({
            'filename': filename,
            'source': 'RAMCO',
            'full_text': text,
            'text_length': len(text),
            'word_count': len(text.split())
        })
        print(f"  ✅ {len(text):,} chars, {len(text.split()):,} words")

print("\n" + "=" * 70)
print(f"✅ Done: {len(extracted_reports)}/{len(pdf_files)} reports")

if extracted_reports:
    print(f"\n📋 Sample:")
    print(extracted_reports[0]['full_text'][:400])

 Installing Poppler...
 Poppler installed!

📖 Extracting text with OCR (5-10 minutes)...

[1/4] Q1Ramcoquarterly2[1].pdf
  📄 Converting to images...
   10 pages converted
  🔍 OCR processing...
    Page 1/10...     Page 2/10...     Page 3/10...     Page 4/10...     Page 5/10...     Page 6/10...     Page 7/10...     Page 8/10...     Page 9/10...     Page 10/10...   ✅ 20,318 chars, 3,395 words

[2/4] RAMCOTheQuarterlyQ[1].pdf
  📄 Converting to images...
   10 pages converted
  🔍 OCR processing...
    Page 1/10...     Page 2/10...     Page 3/10...     Page 4/10...     Page 5/10...     Page 6/10...     Page 7/10...     Page 8/10...     Page 9/10...     Page 10/10...   ✅ 25,213 chars, 4,188 words

[3/4] Q2Ramcoquarterly2[1].pdf
  📄 Converting to images...
   10 pages converted
  🔍 OCR processing...
    Page 1/10...     Page 2/10...     Page 3/10...     Page 4/10...     Page 5/10...     Page 6/10...     Page 7/10...     Page 8/10...     Page 9/10...     Page 10/10...   ✅ 19,537 chars, 3,249 w

In [6]:
# Prepare Documents for Vector Database

print("📝 Preparing documents for RAG system...")
print("=" * 70)

# ============================================================================
# PART 1: Prepare Property Documents
# ============================================================================

print("\n[1/2] Processing Properties...")

property_documents = []

for idx, row in df.iterrows():
    # Create rich text description
    text_parts = []

    if pd.notna(row.get('title')):
        text_parts.append(f"{row['title']}")

    if pd.notna(row.get('property_type')):
        text_parts.append(f"Type: {row['property_type']}")

    # Location
    location_parts = []
    if pd.notna(row.get('district')):
        location_parts.append(row['district'])
    if pd.notna(row.get('governorate')):
        location_parts.append(row['governorate'])
    if location_parts:
        text_parts.append(f"Location: {', '.join(location_parts)}")

    # Price
    if pd.notna(row.get('price_usd')):
        text_parts.append(f"Price: ${row['price_usd']:,.0f}")

    if pd.notna(row.get('size_sqm')):
        text_parts.append(f"Size: {row['size_sqm']} sqm")

    if pd.notna(row.get('price_per_sqm')):
        text_parts.append(f"${row['price_per_sqm']:.2f} per sqm")

    # Layout
    if pd.notna(row.get('bedrooms')):
        text_parts.append(f"{int(row['bedrooms'])} bedrooms")
    if pd.notna(row.get('bathrooms')):
        text_parts.append(f"{int(row['bathrooms'])} bathrooms")

    # Features
    if pd.notna(row.get('features')):
        features_str = str(row['features']).replace("'", "").replace("[", "").replace("]", "")
        text_parts.append(f"Features: {features_str}")

    # Create document
    property_documents.append({
        'id': f"property_{row.get('property_id', idx)}",
        'text': ". ".join(text_parts),
        'type': 'property',
        'metadata': {
            'property_id': str(row.get('property_id', idx)),
            'property_type': str(row.get('property_type', '')),
            'district': str(row.get('district', '')),
            'governorate': str(row.get('governorate', '')),
            'price_usd': float(row['price_usd']) if pd.notna(row.get('price_usd')) else 0,
            'bedrooms': int(row['bedrooms']) if pd.notna(row.get('bedrooms')) else 0,
        }
    })

print(f"✅ Created {len(property_documents)} property documents")
print(f"\n📋 Sample property document:")
print(property_documents[0]['text'][:300] + "...")

# ============================================================================
# PART 2: Prepare Report Documents (Chunked)
# ============================================================================

print(f"\n[2/2] Processing Reports...")

report_documents = []

for report in extracted_reports:
    text = report['full_text']
    filename = report['filename']

    # Chunk into smaller pieces (1000 characters, 200 overlap)
    chunk_size = 1000
    overlap = 200

    start = 0
    chunk_id = 0

    while start < len(text):
        end = start + chunk_size
        chunk_text = text[start:end]

        # Try to break at sentence
        if end < len(text):
            last_period = chunk_text.rfind('.')
            if last_period > chunk_size * 0.6:
                end = start + last_period + 1
                chunk_text = text[start:end]

        # Only add if chunk has meaningful content
        if len(chunk_text.strip()) > 100:
            report_documents.append({
                'id': f"report_{filename}_{chunk_id}",
                'text': chunk_text.strip(),
                'type': 'report',
                'metadata': {
                    'source': 'RAMCO',
                    'filename': filename,
                    'chunk_id': chunk_id
                }
            })
            chunk_id += 1

        start = end - overlap

print(f"✅ Created {len(report_documents)} report chunks")

# ============================================================================
# Summary
# ============================================================================

all_documents = property_documents + report_documents

print("\n" + "=" * 70)
print("📊 SUMMARY:")
print(f"  • Properties: {len(property_documents)} documents")
print(f"  • Reports: {len(report_documents)} chunks")
print(f"  • TOTAL: {len(all_documents)} documents ready for RAG")
print("=" * 70)

📝 Preparing documents for RAG system...

[1/2] Processing Properties...
✅ Created 200 property documents

📋 Sample property document:
Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $305,433. Size: 85 sqm. $3593.33 per sqm. 3 bedrooms. 1 bathrooms. Features: parking, view, elevator...

[2/2] Processing Reports...
✅ Created 121 report chunks

📊 SUMMARY:
  • Properties: 200 documents
  • Reports: 121 chunks
  • TOTAL: 321 documents ready for RAG


In [7]:
import chromadb

print("🗄️  Building Vector Database...")
print("=" * 70)

# Initialize ChromaDB
# Using PersistentClient for local storage, which replaces the deprecated Settings approach
client = chromadb.PersistentClient(path="./chroma_db")

# Create collection for properties
print("\n[1/2] Creating Properties Collection...")
properties_collection = client.get_or_create_collection(
    name="lebanon_properties",
    metadata={"description": "Lebanese real estate properties"}
)

# Add property documents
property_ids = [doc['id'] for doc in property_documents]
property_texts = [doc['text'] for doc in property_documents]
property_metadatas = [doc['metadata'] for doc in property_documents]

properties_collection.add(
    ids=property_ids,
    documents=property_texts,
    metadatas=property_metadatas
)

print(f"✅ Added {len(property_documents)} properties to vector DB")

# Create collection for reports
print("\n[2/2] Creating Reports Collection...")
reports_collection = client.get_or_create_collection(
    name="lebanon_reports",
    metadata={"description": "RAMCO market reports"}
)

# Add report documents
report_ids = [doc['id'] for doc in report_documents]
report_texts = [doc['text'] for doc in report_documents]
report_metadatas = [doc['metadata'] for doc in report_documents]

reports_collection.add(
    ids=report_ids,
    documents=report_texts,
    metadatas=report_metadatas
)

print(f"✅ Added {len(report_documents)} report chunks to vector DB")

# Verify
print("\n" + "=" * 70)
print("✅ VECTOR DATABASE READY!")
print(f"  • Properties collection: {properties_collection.count()} documents")
print(f"  • Reports collection: {reports_collection.count()} documents")
print("=" * 70)

🗄️  Building Vector Database...

[1/2] Creating Properties Collection...


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 67.8MiB/s]


✅ Added 200 properties to vector DB

[2/2] Creating Reports Collection...
✅ Added 121 report chunks to vector DB

✅ VECTOR DATABASE READY!
  • Properties collection: 200 documents
  • Reports collection: 121 documents


In [8]:
# Test Vector Search

def test_search():
    """Test semantic search on both collections"""

    print("🔍 TESTING VECTOR SEARCH")
    print("=" * 70)

    # Test 1: Property Search
    print("\n📍 TEST 1: Property Search")
    print("Query: 'apartment in Achrafieh under 400000'")
    print("-" * 70)

    results = properties_collection.query(
        query_texts=["apartment in Achrafieh under 400000"],
        n_results=3
    )

    for i in range(len(results['ids'][0])):
        print(f"\n{i+1}. {results['ids'][0][i]}")
        meta = results['metadatas'][0][i]
        print(f"   District: {meta.get('district')}")
        print(f"   Price: ${meta.get('price_usd'):,.0f}")
        print(f"   Type: {meta.get('property_type')}")
        print(f"   Bedrooms: {meta.get('bedrooms')}")
        print(f"   Preview: {results['documents'][0][i][:150]}...")

    # Test 2: Report Search
    print("\n\n📊 TEST 2: Market Report Search")
    print("Query: 'Beirut real estate market trends and prices'")
    print("-" * 70)

    results = reports_collection.query(
        query_texts=["Beirut real estate market trends and prices"],
        n_results=3
    )

    for i in range(len(results['ids'][0])):
        print(f"\n{i+1}. {results['ids'][0][i]}")
        meta = results['metadatas'][0][i]
        print(f"   Source: {meta.get('filename')}")
        print(f"   Chunk: {meta.get('chunk_id')}")
        print(f"   Text: {results['documents'][0][i][:200]}...")

    # Test 3: Combined Query
    print("\n\n🔄 TEST 3: Investment Query")
    print("Query: 'good investment properties with parking'")
    print("-" * 70)

    prop_results = properties_collection.query(
        query_texts=["good investment properties with parking"],
        n_results=2
    )

    report_results = reports_collection.query(
        query_texts=["investment advice real estate market"],
        n_results=2
    )

    print("\nFound Properties:")
    for i in range(len(prop_results['ids'][0])):
        meta = prop_results['metadatas'][0][i]
        print(f"  • {meta.get('district')} - ${meta.get('price_usd'):,.0f}")

    print("\nFound Market Insights:")
    for i in range(len(report_results['ids'][0])):
        text = report_results['documents'][0][i][:150]
        print(f"  • {text}...")

    print("\n" + "=" * 70)
    print("✅ Vector search working perfectly!")
    print("🎉 Ready to add AI for intelligent answers!")

# Run tests
test_search()

🔍 TESTING VECTOR SEARCH

📍 TEST 1: Property Search
Query: 'apartment in Achrafieh under 400000'
----------------------------------------------------------------------

1. property_LB00075
   District: Achrafieh
   Price: $423,054
   Type: apartment
   Bedrooms: 1
   Preview: Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $423,054. Size: 171 sqm. $2474.00 per sqm. 1 bedrooms. 3 bathrooms. Featu...

2. property_LB00063
   District: Achrafieh
   Price: $360,441
   Type: apartment
   Bedrooms: 4
   Preview: Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $360,441. Size: 278 sqm. $1296.55 per sqm. 4 bedrooms. 2 bathrooms. Featu...

3. property_LB00026
   District: Achrafieh
   Price: $513,588
   Type: apartment
   Bedrooms: 4
   Preview: Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $513,588. Size: 206 sqm. $2493.15 per sqm. 4 bedrooms. 2 bathrooms. Featu...


📊 TEST 2: Market Report Search
Query: 'Bei

In [9]:
# Load FREE AI Model (TinyLlama)


!pip install -q transformers torch accelerate

from transformers import pipeline
import torch

# Load small, fast, FREE model
generator = pipeline(
    'text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    torch_dtype=torch.float16,
    device_map='auto'
)

print("✅ Model loaded!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded!


In [13]:
# Improved RAG System with Filters

def ask_advisor_improved(question, n_results=5):
    """
    Improved RAG with price filtering and better responses
    """
    print(f"\n{'='*70}")
    print(f"❓ {question}")
    print(f"{'='*70}")

    # Extract price filter from question
    import re
    price_match = re.search(r'under \$?(\d+),?(\d+)', question.lower())
    max_price = None
    if price_match:
        if price_match.group(2):
            max_price = int(price_match.group(1) + price_match.group(2))
        else:
            max_price = int(price_match.group(1)) * 1000
        print(f"💰 Filtering: Properties under ${max_price:,}")

    # Search properties
    print("🔍 Searching properties...")

    # Get more results to filter
    prop_results = properties_collection.query(
        query_texts=[question],
        n_results=20  # Get more to filter
    )

    # Filter by price if needed
    filtered_props = []
    for i in range(len(prop_results['ids'][0])):
        meta = prop_results['metadatas'][0][i]
        price = meta.get('price_usd', 0)

        if max_price is None or price <= max_price:
            filtered_props.append({
                'id': prop_results['ids'][0][i],
                'text': prop_results['documents'][0][i],
                'meta': meta
            })

        if len(filtered_props) >= n_results:
            break

    if not filtered_props:
        print("❌ No properties found matching criteria")
        return

    print(f"✅ Found {len(filtered_props)} matching properties")

    # Search reports
    report_results = reports_collection.query(
        query_texts=[question],
        n_results=2
    )

    # Build context
    context = "PROPERTIES MATCHING YOUR CRITERIA:\n\n"

    for i, prop in enumerate(filtered_props):
        meta = prop['meta']
        context += f"Property {i+1} - {prop['id']}:\n"
        context += f"Location: {meta.get('district')}\n"
        context += f"Price: ${meta.get('price_usd'):,.0f}\n"
        context += f"Details: {prop['text'][:200]}\n\n"

    context += "\nMARKET INSIGHTS:\n"
    for i in range(len(report_results['ids'][0])):
        context += report_results['documents'][0][i][:250] + "\n\n"

    # Create better prompt
    prompt = f"""You are a Lebanese real estate investment advisor.

Question: {question}

{context}

Provide helpful investment advice considering:
- Lebanese market conditions
- Property features and pricing
- Location quality
- Investment potential

Answer (be specific and cite property IDs):"""


    # Generate
    print("🤖 Generating advice...")
    response = generator(
        prompt,
        max_new_tokens=400,
        temperature=0.7,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id
    )

    answer = response[0]['generated_text'].split("<|assistant|>")[-1].strip()

    # Display
    print("\n" + "="*70)
    print("💬 INVESTMENT ADVISOR:")
    print("="*70)
    print(answer[:600])  # Show first 600 chars

    print("\n" + "="*70)
    print("📋 RECOMMENDED PROPERTIES:")
    print("="*70)

    for i, prop in enumerate(filtered_props[:5]):
        meta = prop['meta']
        print(f"\n{i+1}. {prop['id']}")
        print(f"   📍 {meta.get('district')}, {meta.get('governorate')}")
        print(f"   💰 ${meta.get('price_usd'):,.0f}")
        print(f"   🏠 {meta.get('property_type')} - {meta.get('bedrooms')} bed, {meta.get('bathrooms', 'N/A')} bath")

        # Extract features
        text = prop['text'].lower()
        features = []
        if 'parking' in text:
            features.append('parking')
        if 'elevator' in text:
            features.append('elevator')
        if 'view' in text:
            features.append('view')
        if features:
            print(f"   ✨ {', '.join(features)}")

print("✅ Improved RAG System Ready!")

✅ Improved RAG System Ready!


In [14]:
# Test
ask_advisor_improved("What are good apartments in Achrafieh under $400,000?")


❓ What are good apartments in Achrafieh under $400,000?
💰 Filtering: Properties under $400,000
🔍 Searching properties...
✅ Found 5 matching properties
🤖 Generating advice...

💬 INVESTMENT ADVISOR:
You are a Lebanese real estate investment advisor.

Question: What are good apartments in Achrafieh under $400,000?

PROPERTIES MATCHING YOUR CRITERIA:

Property 1 - property_LB00063:
Location: Achrafieh
Price: $360,441
Details: Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $360,441. Size: 278 sqm. $1296.55 per sqm. 4 bedrooms. 2 bathrooms. Features: generator, elevator, view

Property 2 - property_LB00023:
Location: Achrafieh
Price: $356,836
Details: Apartment in Achrafieh. Type: apartment. Location: Achrafieh, Beirut. Price: $356,836. Size: 98 sqm. $3641.18 per

📋 RECOMMENDED PROPERTIES:

1. property_LB00063
   📍 Achrafieh, Beirut
   💰 $360,441
   🏠 apartment - 4 bed, N/A bath
   ✨ elevator, view

2. property_LB00023
   📍 Achrafieh, Beirut
   💰 $356,836
   🏠 

In [15]:
#  Beautiful Web Interface with Gradio

import gradio as gr

def chat_interface(question):
    """Wrapper for Gradio UI"""
    if not question.strip():
        return "Please ask a question about Lebanese real estate!"

    # Extract price filter
    import re
    price_match = re.search(r'under \$?(\d+),?(\d+)', question.lower())
    max_price = None
    if price_match:
        if price_match.group(2):
            max_price = int(price_match.group(1) + price_match.group(2))
        else:
            max_price = int(price_match.group(1)) * 1000

    # Search properties
    prop_results = properties_collection.query(
        query_texts=[question],
        n_results=20
    )

    # Filter by price
    filtered_props = []
    for i in range(len(prop_results['ids'][0])):
        meta = prop_results['metadatas'][0][i]
        price = meta.get('price_usd', 0)

        if max_price is None or price <= max_price:
            filtered_props.append({
                'id': prop_results['ids'][0][i],
                'text': prop_results['documents'][0][i],
                'meta': meta
            })

        if len(filtered_props) >= 5:
            break

    if not filtered_props:
        return "❌ No properties found matching your criteria. Try adjusting your budget or location."

    # Search reports
    report_results = reports_collection.query(
        query_texts=[question],
        n_results=2
    )

    # Build context
    context = "AVAILABLE PROPERTIES:\n\n"
    for i, prop in enumerate(filtered_props):
        meta = prop['meta']
        context += f"{i+1}. {prop['id']} - {meta.get('district')} - ${meta.get('price_usd'):,.0f}\n"
        context += f"{prop['text'][:150]}\n\n"

    context += "\nMARKET CONTEXT:\n"
    for i in range(min(2, len(report_results['ids'][0]))):
        context += report_results['documents'][0][i][:200] + "\n"

    # Generate response
    prompt = f"""<|system|>
You are a Lebanese real estate investment advisor. Give specific recommendations.</s>
<|user|>
{question}

{context}</s>
<|assistant|>"""

    response = generator(
        prompt,
        max_new_tokens=350,
        temperature=0.7,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id
    )

    answer = response[0]['generated_text'].split("<|assistant|>")[-1].strip()

    # Format output
    output = f"## 💬 Investment Advisor\n\n{answer}\n\n---\n\n## 📋 Recommended Properties\n\n"

    for i, prop in enumerate(filtered_props[:5]):
        meta = prop['meta']
        output += f"\n**{i+1}. {prop['id']}**\n"
        output += f"- 📍 Location: {meta.get('district')}, {meta.get('governorate')}\n"
        output += f"- 💰 Price: ${meta.get('price_usd'):,.0f}\n"
        output += f"- 🏠 Type: {meta.get('property_type')} - {meta.get('bedrooms')} bedrooms\n"

        text = prop['text'].lower()
        features = []
        for feat in ['parking', 'elevator', 'view', 'generator', 'balcony']:
            if feat in text:
                features.append(feat)
        if features:
            output += f"- ✨ Features: {', '.join(features)}\n"
        output += "\n"

    return output

# Create interface
interface = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(
        label="Ask Your Real Estate Question",
        placeholder="e.g., Show me 3-bedroom apartments in Beirut under $350,000",
        lines=3
    ),
    outputs=gr.Markdown(label="Investment Advisor Response"),
    title="🏠 Lebanon Real Estate Investment Advisor",
    description="""
    **FREE AI-Powered Real Estate Advisor** for Lebanese Properties

    Ask about:
    - Property recommendations by location and budget
    - Investment analysis and market insights
    - District comparisons
    - Property features and amenities

    *Powered by 200 properties + RAMCO market reports*
    """,
    examples=[
        ["What are the best apartments in Achrafieh under $400,000?"],
        ["Show me properties in Beirut with parking and elevator"],
        ["Compare real estate in Achrafieh vs Byblos"],
        ["What are good investment properties with 3 bedrooms?"],
        ["Tell me about the real estate market trends in Lebanon"]
    ],
    theme=gr.themes.Soft(),
    allow_flagging="never"
)

# Launch
print("🚀 Launching web interface...")
interface.launch(share=True, debug=False)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


🚀 Launching web interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13b982929b69c6e37c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
